In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN, HDBSCAN
from sklearn.datasets import make_blobs
import hdbscan

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D


def plot_sepsis_clusters(dataframe, eps=0.5, min_samples=5, save_path="sepsis_plot.png"):
    """
    Aplica o DBSCAN para identificar clusters de pacientes e compara com o diagnóstico de sepse.

    Parameters:
    - dataframe (pd.DataFrame): O DataFrame contendo os dados.
        - Deve conter a coluna 'diagnostico' indicando sepse (1) ou não (0).
    - eps (float): A distância máxima entre dois pontos para serem considerados vizinhos (parâmetro DBSCAN).
    - min_samples (int): O número mínimo de pontos necessários para formar um cluster (parâmetro DBSCAN).
    - save_path (str): Caminho para salvar a imagem de saída.
    """

    # Verificar se o dataframe possui a coluna 'diagnostico'
    if 'diagnostico' not in dataframe.columns:
        raise KeyError("A coluna 'diagnostico' não está presente no DataFrame.")

    # Separar a coluna 'diagnostico' dos sinais vitais
    features = dataframe.drop(columns=['diagnostico'])
    labels = dataframe['diagnostico']

    # Imputar valores ausentes com a média da coluna
    imputer = SimpleImputer(strategy='mean')
    imputed_data = imputer.fit_transform(features)

    # Normalizar os dados
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(imputed_data)

    # Redução dimensional para 3D com PCA
    pca = PCA(n_components=3)
    reduced_data = pca.fit_transform(normalized_data)

    # Criar um novo DataFrame com os dados reduzidos
    reduced_df = pd.DataFrame(reduced_data, columns=['PC1', 'PC2', 'PC3'])
    reduced_df['diagnostico'] = labels.values

    # Aplicar DBSCAN ao DataFrame reduzido
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    reduced_df['cluster'] = dbscan.fit_predict(reduced_df[['PC1', 'PC2', 'PC3']])

    # Calcular a silhueta média para avaliar a qualidade dos clusters
    if len(set(reduced_df['cluster'])) > 1:  # Verifica se há mais de um cluster
        silhouette_avg = silhouette_score(reduced_df[['PC1', 'PC2', 'PC3']], reduced_df['cluster'])
        print("Silhouette Coefficient:", silhouette_avg)
    else:
        print("Não foi possível calcular o coeficiente de silhueta. Apenas um cluster foi encontrado.")

    # Plotar os resultados em 3D, colorindo os clusters e os pontos sépticos
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3D')

    scatter = ax.scatter(reduced_df['PC1'], reduced_df['PC2'], reduced_df['PC3'],
               c=reduced_df['cluster'], cmap='Set1', label='Clusters')

    # Destacar os pacientes sépticos com um marcador diferente
    ax.scatter(reduced_df[reduced_df['diagnostico'] == 1]['PC1'],
               reduced_df[reduced_df['diagnostico'] == 1]['PC2'],
               reduced_df[reduced_df['diagnostico'] == 1]['PC3'],
               marker='x', color='red', label='Pacientes Sépticos')

    ax.set_title('Detecção de Sepse com DBSCAN (3D)')
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    ax.legend()
    plt.savefig(save_path)
    plt.show()


# Exemplo de uso:
# df = pd.read_csv('seus_dados.csv')
# plot_sepsis_clusters(df)


In [41]:

df = pd.read_csv('df_final.csv')
plot_sepsis_clusters(df)

Silhouette Coefficient: 0.20951154096341212


ValueError: Unknown projection '3D'

<Figure size 1200x800 with 0 Axes>